In [25]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import time
import Levenshtein as lev

In [4]:
forbes_list_2018 = pd.read_csv("file:///Users/tbro/Projects/WikiData/corpedia/Forbes_2000_2018.csv")

In [5]:
forbes_list_2018 = forbes_list_2018[["rank", "name", "country"]]
forbes_list_2018 = forbes_list_2018.sort_values(by="rank")
forbes_list_2018.set_index("rank", drop= True)

,name,country
rank,,
1,ICBC,China
2,China Construction Bank,China
3,JPMorgan Chase,United States
4,Berkshire Hathaway,United States
5,Agricultural Bank of China,China
6,Bank of America,United States
7,Wells Fargo,United States
8,Apple,United States
9,Bank of China,China


In [47]:
def QuerySparql(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

In [48]:
def GenQuery(raw_name): 
    result= """SELECT DISTINCT ?item ?itemLabel ?alias WHERE {?item wdt:P414 ?stock; rdfs:label ?itemLabel; skos:altLabel ?alias FILTER (lang(?itemLabel) = 'en'). FILTER (LANG (?alias) = "en"). FILTER regex (?alias, '^.*""" + raw_name + ".*$').}"
    return result

In [49]:
def GetWikiIDs(query_list, raw_name):
    wiki_id = query_list["item"]["value"].replace("http://www.wikidata.org/entity/","")
    match_name = query_list["itemLabel"]["value"]
    alias = query_list["alias"]["value"]
    if (lev.ratio(match_name.lower(),raw_name.lower())<0.5 or lev.ratio(alias.lower(),raw_name.lower())<0.5):
        return wiki_id, match_name
    else:
        return NA

In [50]:
def GetWikiIDsFromRawName (raw_name):
    query = GenQuery(raw_name)
    result_list = QuerySparql(query)
    result = list(map(GetWikiIDs, result_list,raw_name))
    time.sleep(5)
    return result

In [52]:
raw_name="DDD"
query = GenQuery(raw_name)
result_list = QuerySparql(query)
result = list(map(GetWikiIDs, result_list, raw_name))

HTTPError: HTTP Error 403: Forbidden

In [59]:
print(result)

[('Q26463', 'Industrial and Commercial Bank of China')]


In [53]:
Forbes_2000_raw_names = pd.Series(forbes_list_2018.iloc[:30,1])

In [54]:
Forbes_2000_raw_names.index = Forbes_2000_raw_names

In [55]:
Forbes_2000_IDs = Forbes_2000_raw_names.apply(GetWikiIDsFromRawName)

HTTPError: HTTP Error 403: Forbidden

In [46]:
Forbes_2000_IDs

name
Royal Dutch Shell                            [(Q154950, Royal Dutch Shell)]
Toyota Motor                                             [(Q53268, Toyota)]
ExxonMobil                                                               []
Samsung Electronics       [(Q20718, Samsung Electronics), (Q20718, Samsu...
AT&T                                                       [(Q35476, AT&T)]
Volkswagen Group                                                         []
HSBC Holdings                                    [(Q190464, HSBC Holdings)]
Verizon Communications             [(Q467752, Verizon), (Q467752, Verizon)]
BNP Paribas                                        [(Q499707, BNP Paribas)]
Microsoft                                              [(Q2283, Microsoft)]
Name: name, dtype: object